<a href="https://colab.research.google.com/github/mapmatters/coursera-nlp/blob/main/nlp_class_1_Week2_nb02_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/coursera/nlp')
!ls

Mounted at /content/drive
'nlp class 1 - Week1 - nb01'
'nlp class 1 - Week1 - nb02'
'nlp class 1 - Week1 - nb03'
'nlp class 1 - Week1 - nb04 - assignment'
'nlp class 1 - Week2 - nb01'
'nlp class 1 - Week2 - nb02 - assignment'
 py
 __pycache__


In [ ]:
# from utils import process_tweet, lookup
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd

In [ ]:
import sys
sys.path.append('py')
from utils import process_tweet, lookup

In [13]:
nltk.download('stopwords')
nltk.download('twitter_samples')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [14]:
# add folder, tmp2, from our local workspace containing pre-downloaded corpora files to nltk's data path
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [15]:
# get the sets of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# split the data into two pieces, one for training and one for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# avoid assumptions about the length of all_positive_tweets
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

## Part 1 - Process the Data

In [36]:
def count_tweets(result, tweets, ys):
    '''
    Input:
        result: a dictionary that will be used to map each pair to its frequency
        tweets: a list of tweets
        ys: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Output:
        result: a dictionary mapping each pair to its frequency
    '''
#     result = {}
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            # define the key, which is the word and label tuple
            pair = (word, y)

            # if the key exists in the dictionary, increment the count
            if pair in result:
                result[pair] += 1

            # else, if the key is new, add it to the dictionary and set the count to 1
            else:
                result[pair] = 1
    ### END CODE HERE ###

    return result

In [37]:
# test
result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

{('happi', 1): 1, ('sad', 0): 1, ('tire', 0): 2, ('trick', 0): 1}

## Part 2 - Train model

In [18]:
# build the freqs dictionary
freqs = count_tweets({}, train_x, train_y)

In [34]:
# train_naive_bayes
def train_naive_bayes(freqs, train_x, train_y):
  """
  V
  freq_pos, freq_neg
  N_pos, N_neg
  D_pos, D_neg
  logprior
  loglikelihood
  """
  loglikelihood = {}
  logprior = 0

  freqs_keys = list(freqs.keys())
  # V
  words = []
  for key in freqs_keys:
    words.append(key[0])
  words = set(words)
  V = len(words)
  
  # freq_pos, freq_neg of each word
  df = pd.DataFrame(columns = ["word","pos","neg"])
  x = np.zeros((V, 3)) 
  for word in words:
    # print(pair)
    row = [word, freqs.get((word,1.0), 0), freqs.get((word,0.0), 0)]
    df = df.append(pd.Series(row, index=df.columns), ignore_index=True)
  
  # N_pos, N_neg
  N_pos = sum(df.pos)
  N_neg = sum(df.neg)
  
  # D_pos, D_neg
  D_pos = np.sum(np.where(train_y==1, 1, 0))
  D_neg = np.sum(np.where(train_y==0, 1, 0))

  # logprior
  logprior = np.log(D_pos) - np.log(D_neg)

  # loglikelihood
  def f(x1, x2):
    p1 = np.log((x1+1)/(N_pos+V))
    p2 = np.log((x2+1)/(N_neg+V))
    return p1 - p2
  df['llhood'] = df.apply(lambda x: f(x.pos, x.neg), axis=1)
  logliklihood = df[['word','llhood']].set_index("word").to_dict()['llhood']

  return logprior, logliklihood

In [ ]:
# from template
# def train_naive_bayes(freqs, train_x, train_y):
#     '''
#     Input:
#         freqs: dictionary from (word, label) to how often the word appears
#         train_x: a list of tweets
#         train_y: a list of labels correponding to the tweets (0,1)
#     Output:
#         logprior: the log prior. (equation 3 above)
#         loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
#     '''
#     loglikelihood = {}
#     logprior = 0

#     ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

#     # calculate V, the number of unique words in the vocabulary
#     vocab = set([pair[0] for pair in freqs.keys()])
#     V = len(vocab)

#     # calculate N_pos and N_neg
#     N_pos = N_neg = 0
#     for pair in freqs.keys():
#         # if the label is positive (greater than zero)
#         if pair[1] > 0:

#             # Increment the number of positive words by the count for this (word, label) pair
#             N_pos += freqs[pair]

#         # else, the label is negative
#         else:

#             # increment the number of negative words by the count for this (word,label) pair
#             N_neg += freqs[pair]

#     # Calculate D, the number of documents
#     D = len(train_y)

#     # Calculate D_pos, the number of positive documents (*hint: use sum(<np_array>))
#     D_pos = np.sum(np.where(train_y==1, 1, 0))

#     # Calculate D_neg, the number of negative documents (*hint: compute using D and D_pos)
#     D_neg = np.sum(np.where(train_y==0, 1, 0))

#     # Calculate logprior
#     logprior = np.log(D_pos) - np.log(D_neg)

#     # For each word in the vocabulary...
#     for word in vocab:
#         # get the positive and negative frequency of the word
#         freq_pos = freqs.get((word,1.0), 0)
#         freq_neg = freqs.get((word,0.0), 0)

#         # calculate the probability that each word is positive, and negative
#         p_w_pos = np.log((freq_pos+1)/(N_pos+V))
#         p_w_neg = np.log((freq_neg+1)/(N_neg+V))

#         # calculate the log likelihood of the word
#         loglikelihood[word] = p_w_pos - p_w_neg

#     ### END CODE HERE ###

#     return logprior, loglikelihood

In [35]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

0.0
9089


## Part 3 - Test your naive bayes
$$ p = logprior + \sum_i^N (loglikelihood_i)$$

In [40]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
  """
  input: tweet(string), logprior, loglikelihood
  output: p (sum of all the likelihood of each word in the tweet)
  """
  word_1 = process_tweet(tweet)
  p = 0
  p += logprior
  
  for word in word_1:
    if word in loglikelihood:
      p += loglikelihood[word]

  return p

In [41]:
# Experiment with your own tweet.
my_tweet = 'She smiled.'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is 1.5740278623499178


In [53]:
# test model
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
  """
  input
    test_x: A list of tweets
    test_y: label
    logprior
    loglikelihood
  output
    accuracy
  """
  accuracy = 0

  y_hats = []
  for tweet in test_x:
    if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
      y_hat_i = 1
    else:
      y_hat_i = 0
    y_hats.append(y_hat_i)

  # error is the average of the absolute values of the differences between y_hats and test_y
  error = sum(np.asarray(y_hats) != test_y)/len(test_y)

  # Accuracy is 1 minus the error
  accuracy = 1 - error

  ### END CODE HERE ###

  return accuracy


In [54]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.9940


In [55]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    # print( '%s -> %f' % (tweet, naive_bayes_predict(tweet, logprior, loglikelihood)))
    p = naive_bayes_predict(tweet, logprior, loglikelihood)
#     print(f'{tweet} -> {p:.2f} ({p_category})')
    print(f'{tweet} -> {p:.2f}')

I am happy -> 2.15
I am bad -> -1.29
this movie should have been great. -> 2.14
great -> 2.14
great great -> 4.28
great great great -> 6.41
great great great great -> 8.55


## Part 4 - Filter word by ratio of positive to negative counts

In [66]:
def get_ratio(freqs, word):
    '''
    Input:
        freqs: dictionary containing the words
        word: string to lookup

    Output: a dictionary with keys 'positive', 'negative', and 'ratio'.
        Example: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # use lookup() to find positive counts for the word (denoted by the integer 1)
    pos_neg_ratio['positive'] = lookup(freqs, word, 1)

    # use lookup() to find negative counts for the word (denoted by integer 0)
    pos_neg_ratio['negative'] = lookup(freqs, word, 0)

    # calculate the ratio of positive to negative counts for the word
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1)/ (pos_neg_ratio['negative'] + 1)
    ### END CODE HERE ###
    return pos_neg_ratio

In [58]:
get_ratio(freqs, 'happi')

{'negative': 18, 'positive': 161, 'ratio': 8.944444444444445}

In [81]:
def get_words_by_threshold(freqs, label, threshold):
  """
  input
    freqs: dictionary of words
    label: 1 for positive, 0 for negative
    threshold: cutoff
  output
    word_set: information of positive count, negative count, ratio of pos/neg (result of get_ratio())
  """
  word_list = {}

  for key in freqs.keys():
    word, _ = key
    pos_neg_ratio = get_ratio(freqs, word)

    if label == 1 and pos_neg_ratio['ratio'] > threshold:
      word_list[word] = pos_neg_ratio
    elif label == 0 and pos_neg_ratio['ratio'] < threshold:
      word_list[word] = pos_neg_ratio

  return word_list

In [82]:
get_words_by_threshold(freqs, label=0, threshold=0.05)

{'26': {'negative': 20, 'positive': 0, 'ratio': 0.047619047619047616},
 ':(': {'negative': 3663, 'positive': 1, 'ratio': 0.0005458515283842794},
 ':-(': {'negative': 378, 'positive': 0, 'ratio': 0.002638522427440633},
 '>:(': {'negative': 43, 'positive': 0, 'ratio': 0.022727272727272728},
 'beli̇ev': {'negative': 35, 'positive': 0, 'ratio': 0.027777777777777776},
 'justi̇n': {'negative': 35, 'positive': 0, 'ratio': 0.027777777777777776},
 'wi̇ll': {'negative': 35, 'positive': 0, 'ratio': 0.027777777777777776},
 '♛': {'negative': 210, 'positive': 0, 'ratio': 0.004739336492890996},
 '》': {'negative': 210, 'positive': 0, 'ratio': 0.004739336492890996},
 'ｍｅ': {'negative': 35, 'positive': 0, 'ratio': 0.027777777777777776},
 'ｓｅｅ': {'negative': 35, 'positive': 0, 'ratio': 0.027777777777777776}}

In [83]:
get_words_by_threshold(freqs, label=1, threshold=10)

{':)': {'negative': 2, 'positive': 2847, 'ratio': 949.3333333333334},
 ':-)': {'negative': 0, 'positive': 543, 'ratio': 544.0},
 ':D': {'negative': 0, 'positive': 498, 'ratio': 499.0},
 ':p': {'negative': 0, 'positive': 103, 'ratio': 104.0},
 ';)': {'negative': 0, 'positive': 22, 'ratio': 23.0},
 'arriv': {'negative': 4, 'positive': 57, 'ratio': 11.6},
 'bam': {'negative': 0, 'positive': 44, 'ratio': 45.0},
 'blog': {'negative': 0, 'positive': 27, 'ratio': 28.0},
 'commun': {'negative': 1, 'positive': 27, 'ratio': 14.0},
 'fav': {'negative': 0, 'positive': 11, 'ratio': 12.0},
 'fback': {'negative': 0, 'positive': 26, 'ratio': 27.0},
 'flipkartfashionfriday': {'negative': 0, 'positive': 16, 'ratio': 17.0},
 'followfriday': {'negative': 0, 'positive': 23, 'ratio': 24.0},
 'glad': {'negative': 2, 'positive': 41, 'ratio': 14.0},
 "here'": {'negative': 0, 'positive': 20, 'ratio': 21.0},
 'influenc': {'negative': 0, 'positive': 16, 'ratio': 17.0},
 'pleasur': {'negative': 0, 'positive': 10, 

# Part 5 - Error analysis

In [84]:
print("Truth predicted tweet")
for x, y in zip(test_x, test_y):
  y_hat = naive_bayes_predict(x, logprior, loglikelihood)
  if y != (np.sign(y_hat) > 0):
    print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Truth predicted tweet
1	0.00	b''
1	0.00	b'truli later move know queen bee upward bound movingonup'
1	0.00	b'new report talk burn calori cold work harder warm feel better weather :p'
1	0.00	b'harri niall 94 harri born ik stupid wanna chang :D'
1	0.00	b''
1	0.00	b''
1	0.00	b'park get sunlight'
1	0.00	b'uff itna miss karhi thi ap :p'
0	1.00	b'hello info possibl interest jonatha close join beti :( great'
0	1.00	b'u prob fun david'
0	1.00	b'pat jay'
0	1.00	b'whatev stil l young >:-('


In [85]:
np.sign(-0.01) > 0

False